<a href="https://colab.research.google.com/github/VladYourich/Face_reco_for_LizaAlert/blob/main/%D0%92%D0%BB%D0%B0%D0%B4%D0%9A%D0%BE%D1%81%D0%BC%D1%8B%D0%BB%D0%B5%D0%B2_%D0%A0%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%9B%D0%B8%D1%86_%D0%B3%D1%80%D1%83%D0%BF%D0%BF%D0%BE%D0%B2%D1%8B%D0%B5%D0%A4%D0%BE%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition

# Настройки

In [ ]:
dburl = 'https://drive.google.com/file/d/1gESdeTtOF_T70Ol7BKL1qLoQEiSEW11w/view?usp=sharing' # la.db
!gdown --id '1gESdeTtOF_T70Ol7BKL1qLoQEiSEW11w'
group_img_url = 'https://drive.google.com/file/d/1UVt8YXtz8J9Wf5UKBSWyYN-S-jaeE6p7/view?usp=sharing'
!gdown --id '1UVt8YXtz8J9Wf5UKBSWyYN-S-jaeE6p7'
one_img_url = 'https://drive.google.com/file/d/1Q5mnm6Ls2qiT6d_L1IPVbOojiEW8GGy7/view?usp=sharing'
!gdown --id '1Q5mnm6Ls2qiT6d_L1IPVbOojiEW8GGy7'
#https://drive.google.com/file/d/1Y4AH8ARsZbu1YPpNmbFXAGQq-T7uy_6_/view?usp=sharing

one_img_url = 'https://drive.google.com/file/d/1Y4AH8ARsZbu1YPpNmbFXAGQq-T7uy_6_/view?usp=sharing'
!gdown --id '1Y4AH8ARsZbu1YPpNmbFXAGQq-T7uy_6_'
# https://drive.google.com/file/d/1NxyqVUS4dmzKxNyCfbmbMqUN_G39Xlns/view?usp=sharing
!gdown --id '1NxyqVUS4dmzKxNyCfbmbMqUN_G39Xlns'
# https://drive.google.com/file/d/1tW1fmquxpBt4xKlwjCdiC7s0K_MyVw5q/view?usp=sharing
!gdown --id '1tW1fmquxpBt4xKlwjCdiC7s0K_MyVw5q'

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, BLOB, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.types import TypeDecorator, PickleType
# from enveronments import fill_callnames, СALLSIGN_ID, СALLSIGN_NAME
import numpy as np
import pickle
import face_recognition
import matplotlib.pyplot as plt

In [ ]:
Base = declarative_base()

class PickleTypeDecorator(TypeDecorator):
    impl = BLOB
    cache_ok = True
    def process_bind_param(self, value, dialect):
        if value is not None:
            return pickle.dumps(value)

    def process_result_value(self, value, dialect):
        if value is not None:
            return pickle.loads(value)

class Callsign(Base):
    __tablename__ = 'callsign'

    id = Column(Integer, primary_key=True)
    callname = Column(String, unique=True, nullable=False)

class Vector(Base):
    __tablename__ = 'vectors'

    id = Column(Integer, primary_key=True)
    vector = Column(PickleTypeDecorator)
    #vector = Column(BLOB, nullable=False)
    #img = Column(BLOB, unique=True, nullable=True)
    call_id = Column(Integer, ForeignKey('callsign.id'))
    call = relationship('Callsign', back_populates='vectors')

Callsign.vectors = relationship('Vector', order_by=Vector.call_id, back_populates='call')

def sess():
    # Создание сессии для работы с базой данных
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

def add_vector(id, callvector): #, callname

    ses = sess()

    # Добавление записей в таблицы
    #call = Callsign(callname=callname)
    vector = Vector(call_id=id, vector=callvector) #, call=call

    ses.add_all([vector]) #call,
    ses.commit()

def add_many_vectors(items):
    ses = sess()
    # Добавление записей списком. Пример содержимого items
    # data_to_add = [
    #     {'callname': 'ГБ', 'vector': [0.7, 0.8, 0.9]},
    #     {'callname': 'Физрук', 'vector': [1.0, 1.1, 1.2]},
    # ]

    for data in items:
        call_id = get_id_for_calname(data['callname'])
        vector = Vector(vector=data['vector'], id=call_id)
        ses.add_all([vector])

    ses.commit()

def add_items_to_callsign(items):
    ses = sess()
    # Добавление записей списком. Пример содержимого items
    # data_to_add = [
    #     {'id': 1,'callname': 'ГБ'},
    #     {'id': 2,'callname': 'ВК74'},
    # ]

    for data in items:
        call = Callsign(id=data['id'], callname=data['callname'])
        ses.add_all([call])

    ses.commit()

def get_vectors(callid) -> list:
    # Запрос к базе для получения списка значений vector с фильтром по callname
    #global СALLSIGN_ID
    ses = sess()

    vectors_for_callname = (
        ses.query(Vector.vector)
        #.join(Vector.call)
        .filter(Vector.call_id == callid)
        .all()
    )
    list_vec = []
    for i in vectors_for_callname:
        qq = i[0][0]
        list_vec.append(qq)
    #print(f'Vectors for callname {СALLSIGN_ID[callid]}: {list_vec}')
    return list_vec


def get_id_for_calname(callname) -> int:
    ses = sess()
    # Запрос к базе для получения всех позывных call_id
    result = (ses.query(Callsign.id).filter(Callsign.callname == callname).one())
    return result.id

# Создание базы данных
engine = create_engine('sqlite:///la.db', echo=False)
Base.metadata.create_all(bind=engine)

In [ ]:
def get_calnames_dict() -> dict:
    ses = sess()
    # Запрос к базе для получения всех позывных call_id
    result = ses.query(Callsign.callname, Callsign.id).all()
    names_id = {key:value for key, value in result}
    id_names = {value:key for key, value in result}
    return names_id, id_names

CALL_NAME_ID, CALL_ID_NAME = get_calnames_dict()

In [ ]:
def get_base_vectors() -> dict:
    vec = {}
    for name, id in CALL_NAME_ID.items():
        #print(name, id)
        vec[name] = get_vectors(id)
    return vec

VECTORS = get_base_vectors()  # словарь со Всеми векторами всех поисковиков

# ЭКСПЕРИМЕНТЫ

In [ ]:
# Путь к групповой фотографии
image_path = "/content/dsc09573.jpg"

## № 1

In [ ]:
image = face_recognition.load_image_file(image_path)

face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)

height, width = image.shape[:2]
plt.figure(figsize=(width/100, height/100))

plt.imshow(image, extent=[0, width, height, 0])
plt.axis('off')

for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    # Отрисовка рамок вокруг найденных лиц
    plt.plot([left, right, right, left, left], [top, top, bottom, bottom, top], color='red')

    for name, known_encoding in VECTORS.items():
        # Вариант с распознаванием по списку векторов
        matches = face_recognition.compare_faces(known_encoding, face_encoding)

        if True in matches:
            #plt.text(left, top, name, fontsize=4, bbox=dict(facecolor='red', alpha=0.2))
            plt.text(left, top + 10, str(name), color='blue', fontsize=16)

plt.show()


## № 2

In [ ]:
import face_recognition
import matplotlib.pyplot as plt

In [ ]:
# Путь к групповой фотографии
image_path = "/content/photo_1674@10-06-2023_13-11-13.jpg"

In [ ]:
# Загрузка групповой фотографии
image = face_recognition.load_image_file(image_path)

# Поиск лиц на фотографии
face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)

# Визуализация найденных лиц
height, width = image.shape[:2]
plt.figure(figsize=(width/100, height/100))

plt.imshow(image, extent=[0, width, height, 0])
plt.axis('off')

# Словарь для хранения соответствий порядкового номера и имени
index_to_name = {}

for i, (top, right, bottom, left) in enumerate(face_locations):
    # Отрисовка рамок вокруг найденных лиц
    plt.plot([left, right, right, left, left], [top, top, bottom, bottom, top], color='red')

    # Поиск совпадений в базе векторов
    found_match = False
    for name, known_encoding in VECTORS.items():
        matches = face_recognition.compare_faces(known_encoding, face_encodings[i])
        if True in matches:
            # Отображение имени человека
            plt.text(left, top, str(i+1), fontsize=12, bbox=dict(facecolor='red', alpha=0.5))
            index_to_name[i+1] = name
            found_match = True
            break

    if not found_match:
        plt.text(left, top, str(i+1), fontsize=12, bbox=dict(facecolor='green', alpha=0.1))
        index_to_name[i+1] = "Unknown"

plt.show()

# Вывод словаря с соответствиями порядкового номера и имени
print("Соответствия порядкового номера и имени:")
for key, value in index_to_name.items():
    print(f"{key}: {value}")


## № 3

In [ ]:
image = face_recognition.load_image_file(image_path)
# Поиск лиц на фотографии
face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)
# Визуализация найденных лиц
height, width = image.shape[:2]
plt.figure(figsize=(width/100, height/100))
plt.imshow(image, extent=[0, width, height, 0])
plt.axis('off')
# Словарь для хранения соответствий порядкового номера и имени
index_to_name = {}

for i, (top, right, bottom, left) in enumerate(face_locations):
    # Отрисовка рамок вокруг найденных лиц
    plt.plot([left, right, right, left, left], [top, top, bottom, bottom, top], color='red')
    # Поиск совпадений в базе векторов
    found_match = False
    for name, known_encodings_list in VECTORS.items():
        matches_count = 0
        for known_encoding in known_encodings_list:
            match = face_recognition.compare_faces([known_encoding], face_encodings[i])[0]
            if match:
                matches_count += 1

        if matches_count > 1:
            # Отображение имени человека
            plt.text(left, bottom, str(i+1), fontsize=15, bbox=dict(facecolor='red', alpha=0.3))
            index_to_name[i+1] = (name, matches_count, len(known_encodings_list))
            found_match = True
            break

    if not found_match:
        plt.text(left, top, str(i+1), fontsize=5, bbox=dict(facecolor='blue', alpha=0.8))
        #index_to_name[i+1] = ("Unknown", 0, 0)

plt.show()

# Вывод словаря с соответствиями порядкового номера и имени
print("Соответствия порядкового номера и имени:")
for key, value in index_to_name.items():
    name, match_count, total_count = value
    print(f"{key}: {name} - {match_count} из {total_count}")


In [ ]:
import plotly.graph_objects as go

# Загрузка изображения
image = face_recognition.load_image_file(image_path)

# Поиск лиц на фотографии
face_locations = face_recognition.face_locations(image)
face_encodings = face_recognition.face_encodings(image, face_locations)

# Создание фигуры для визуализации
fig = go.Figure()

# Добавление изображения на график
fig.add_layout_image(
    source=image,
    x=0,
    sizex=image.shape[1],
    y=0,
    sizey=image.shape[0],
    xref="x",
    yref="y",
    opacity=1,
    layer="below"
)

# Создание списка для хранения соответствий порядкового номера и имени
index_to_name = {}

for i, (top, right, bottom, left) in enumerate(face_locations):
    # Определение координат прямоугольника
    x_coordinates = [left, right, right, left, left]
    y_coordinates = [top, top, bottom, bottom, top]

    # Добавление прямоугольника как полигона на график
    fig.add_trace(go.Scatter(
        x=x_coordinates,
        y=y_coordinates,
        mode='lines',
        line=dict(color='red', width=1),
        fill='toself',  # Закрашиваем прямоугольник
        fillcolor='rgba(255, 0, 0, 0.3)'  # Цвет и прозрачность заливки
    ))

    for name, known_encodings_list in VECTORS.items():
        transact = 0
        matches_count = 0
        for known_encoding in known_encodings_list:
            match = face_recognition.compare_faces([known_encoding], face_encodings[i])[0]
            if match:
                matches_count += 1

        if (transact == len(known_encodings_list)) and (matches_count >= len(known_encodings_list)//2) :
            # Отображение имени человека
            # plt.text(left, bottom, str(i+1), fontsize=15, bbox=dict(facecolor='red', alpha=0.3))
            # Добавление аннотации
            fig.add_annotation(
                x=(left + right) / 2,
                y=top - 10,
                text=f"Face {i+1}",
                showarrow=False,
                font=dict(color='black', size=14),
                bgcolor='rgba(255, 255, 255, 0.3)',  # Цвет и прозрачность фона аннотации
                bordercolor='black',  # Цвет границы аннотации
                borderwidth=1,  # Ширина границы аннотации
                borderpad=4,  # Расстояние от границы аннотации до текста
            )

            index_to_name[i+1] = (name, matches_count, len(known_encodings_list))
            found_match = True
            # break

# Настройка макета
fig.update_layout(
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    showlegend=False,
    width=image.shape[1],
    height=image.shape[0]
)

# Отображение графика
fig.show()

In [ ]:
import plotly.graph_objects as go

# Задаем координаты прямоугольника
left, right, top, bottom = 100, 200, 100, 200

# Создаем фигуру
fig = go.Figure()

# Добавляем прямоугольник как трассировку
fig.add_trace(go.Scatter(
    x=[left, right, right, left, left],
    y=[top, top, bottom, bottom, top],
    mode='lines',
    line=dict(color='red', width=1),
    fill='toself',  # Закрашиваем прямоугольник
    fillcolor='rgba(255, 0, 0, 0.3)'  # Устанавливаем цвет и прозрачность заливки
))

# Добавляем аннотацию
fig.add_annotation(
    x=(left + right) / 2,
    y=top + 30,
    text="Пример аннотации",
    showarrow=False,
    font=dict(color='black', size=14),
    bgcolor='rgba(255, 255, 255, 0.7)',  # Цвет и прозрачность фона аннотации
    bordercolor='black',  # Цвет границы аннотации
    borderwidth=1,  # Ширина границы аннотации
    borderpad=4,  # Расстояние от границы аннотации до текста
    ax=0,  # Сдвиг по оси X относительно точки аннотации
    ay=-40  # Сдвиг по оси Y относительно точки аннотации
)

# Настройки макета
fig.update_layout(
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    showlegend=False,
    width=800,
    height=600
)

# Отображаем график
fig.show()